# OGGM v1.6 read-in

Sandbox notebook to read in new OGGM v1.6 output generated by Lilian Schuster.  Note that we expect there will be even newer v1.6.1 output forthcoming, but the format should be consistent. 

2 Jun 2023 | EHU

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from cycler import cycler
import datetime 

### Define the case we'll study
We will need to make some consistent selections below: which basin, which SSP, which GCM to examine the glacier models' output.

In [ ]:
alpine_basins = {'RHINE': '6242',
                 'RHONE': '6243',
                 'PO': '6241'} ## GRDC Major River Basin identifiers for the 3 alpine basins we can study

test_basin = alpine_basins['RHONE'] 

all_ssps = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
which_ssp = all_ssps[2]   #specifying the SSP

#All of the climate models used in GloGEM
modelnames = ['BCC-CSM2-MR','CAMS-CSM1-0','CESM2','CESM2-WACCM','EC-Earth3','EC-Earth3-Veg','FGOALS-f3-L','GFDL-ESM4',
              'INM-CM4-8','INM-CM5-0','MPI-ESM1-2-HR','MRI-ESM2-0']
which_model = modelnames[0] ## specify one model (for GloGEM format)

# PyGEM has 12 GCM forcings as well; TODO: confirm they are in the same order as above.  Until then, use BCC-CSM2-MR
which_gcm = 1 ## applies to PyGEM

# TODO: specify for OGGM format?

In [ ]:
## define local file paths
fpath_oggm = '/Volumes/GoogleDrive/My Drive/Runoff-intercomparison/OGGM/lschuster/{}'.format(test_basin)

### Read in list of glaciers in our basin of interest
Erik Holmgren provided a list of glaciers (by RGI ID) in each major river basin, stored as a JSON file.  Use the function he defined in gha.utils to reference that list and select all the IDs that fall within an example basin.

In [ ]:
import json

def select_glaciers_json(basin='all'):
    '''
    Select glaciers within a basin by MRBID from a json-file,
    which is stored in the data directory.

    Args:
    -----
    basin: str
        String of MRBID or 'all'.

    Returns:
    --------
    If basin is 'all' a list of all relevant glaciers is returned, for
    initiating glacier simulations. If basin is a MRBID the list of glaciers
    within that basin is returned.
    
    Copy of a function written by Erik Holmgren (2022) in holmgren_gha.utils
    '''

    # fpath = './data/rgi_ids_per_basin.json'
    fpath = '/Users/lizz/Documents/Research/Runoff-intercomparison/msc_thesis-multi_gcm/code/data/rgi_ids_per_basin.json' ## correct for local run
    with open(fpath) as f:
        basin_dict = json.load(f)

    if basin.lower() != 'all':
        glacier_list = basin_dict[basin]
    else:
        glacier_list = list(itertools.chain.from_iterable(basin_dict.values()))

    return glacier_list

In [ ]:
gl = select_glaciers_json(test_basin)

### Add OGGM output
Pull in data from Erik Holmgren's simulations to compare.


In [ ]:
with xr.open_mfdataset(f'{fpath_oggm}/*.nc') as ds:
    ds_r = ds.runoff.load()
    ds_v = ds.volume.load()

In [ ]:
total_runoff_245 = 1e-12 * ds_r.sel(scenario=which_ssp, gcm=which_model).sum(dim='rgi_id')[:-1]
## remove final value, which is an unphysical plummet to 0 at 2100

In [ ]:
fig, ax = plt.subplots()
total_runoff_245.plot()
ax.set(ylabel='Annual runoff [Gt]', xlabel='Year',
      title='Rhone basin runoff simulated with OGGM, forced by {}, {} \n {}'.format(
      which_model, which_ssp, datetime.date.today()));

In [ ]:
fig, ax = plt.subplots()
ds_v.sel(scenario=which_ssp, gcm=which_model).sum(dim='rgi_id')[:-1].plot()
ax.set(ylabel='Annual runoff [Gt]', xlabel='Year',
      title='Rhone basin runoff simulated with OGGM, forced by {}, {} \n {}'.format(
      which_model, which_ssp, datetime.date.today()));